In [0]:
# 1 
# Importing all the required libraries 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer

In [0]:
# Configuring Google colab to pick the input files from Google Drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 4.9MB/s 


In [0]:
# Get the Stemmed Text CSV file
link = "https://drive.google.com/open?id=10ZYvHgQUs7YD0mmSy4jow0SZYWO-6r9l" # link to google drive Stemmed Text.csv
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
_stem_text_df = pd.read_csv('Filename.csv')
# Remove the files without any Eglish text (feature vector)
_stem_text_df["English"] = _stem_text_df["Text"].str.contains('english version document avail sinc includ english special edit', regex=True)
#_stem_text_df['English'].value_counts()

In [0]:
# Get the Eurovoc Labels from Google Drive
link = "https://drive.google.com/open?id=1l_LgY8OKrW4tCuCg17kI-6c0qN1ZiYBp" # link to google drive Stemmed Text.csv
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
_label_df = pd.read_csv("Filename.csv")    

Insert Graphs from Rabi's code here

In [0]:
# Reading from labels file and transposing them. One row each for a file. 
# Transpose using pivot table. So, we have as many columns as distinct labels. Value under a label shows whether the document has the label or not.
_label_df['pivotcol'] = 1
_lbl_trnsps_df = pd.pivot_table(_label_df,values='pivotcol',index=['Filename'],columns=['Eurovoc_Label'],aggfunc=np.sum, fill_value=0)

In [0]:
# Get the list of labels into a column
_label_df = _label_df.groupby(['Filename'])['Eurovoc_Label'].apply(list).reset_index()

In [0]:
# merge the features and labels transposed 
df_for_stemming = _stem_text_df[_stem_text_df['English']==False]
_stem_text_df = df_for_stemming.drop(columns='English')
_array_df = pd.merge(_stem_text_df, _lbl_trnsps_df, on='Filename')
_array_df = _array_df.drop(columns='Filename')

In [0]:
Y_labels = pd.merge(_stem_text_df, _label_df, on='Filename')
Y_labels = Y_labels.drop(columns = ["Text","Filename"])

In [0]:
Y_labels

In [0]:
corr = _array_df.drop(columns = "Text")
# checking the correlation between the labels
corr = corr.corr()

In [0]:
#### for dropping the correlated labels 

col = corr.columns
cols_to_drop = []
k =0
num_classes = 0
for i in range(0,len(corr)):
  for j in range(0,len(corr)):
   # try: 
      if((float(corr[col[i]][j])>.9) and  (col[j] != i) and i<j):
        if(sum(_array_df[col[i]])) < sum(_array_df[col[j]]):
          cols_to_drop.append(col[i])
        else:
          cols_to_drop.append(col[j])
        
      #print(corr[i][j])
      #print("feture1",i,"feature 2",col[j])
        num_classes = num_classes +1
        print(num_classes)
    #except:
     # print(_array_df[col[i]],_array_df[col[j]])
     # continue

# we have 128 labels ( .9 corr)which are highly correlated. possibly due to subcategories. Hence learning subcategory is same as learning the main class. 
# we have 183 labels ( .8 corr)
# we havev 153 labels which have just 1 dataset and they co-occur. 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128


Correlated matrix

In [0]:
cols_to_drop

['Fisheries',
 'Animal health and zootechnics',
 'Approximation of laws and health measures',
 'Approximation of laws and health measures',
 'Customs Union and free movement of goods',
 'Individual tax exemptions',
 'Market access',
 'Market operation',
 'Reliefs from duty',
 'Shipping',
 'Victualling procedures',
 'Euro-Arab cooperation',
 'General provisions',
 'Judicial cooperation in civil matters',
 'Dominica',
 'cost-benefit analysis',
 'cooperation procedure',
 'Corporation tax',
 'Company law',
 'Elimination of double taxation',
 'Corporation tax',
 'Elimination of double taxation',
 'disaffection of young people',
 'Individual tax exemptions',
 'Market access',
 'Market operation',
 'Reliefs from duty',
 'Shipping',
 'Victualling procedures',
 'Elimination of double taxation',
 'legal hearing',
 'constitutional law',
 'Nuclear energy',
 'Environment, consumers and health protection',
 'Ethiopia',
 'education budget',
 'national election',
 'Lomé Convention',
 'Judicial coopera

In [0]:
_array_df = _array_df.drop(columns = cols_to_drop)

In [0]:
### List drops of correlated items from list

for j in cols_to_drop:
  for i in Y_labels["Eurovoc_Label"]:
  
    if(j in i):
      print(i)
      print("removing",j)
      i.remove(j)
      print(i)
Y = Y_labels

['Fisheries', 'External relations', 'Agreements with non-member countries', 'External relations', 'Bilateral agreements with non-member countries', 'African, Caribbean and Pacific (ACP) Group of States']
removing Fisheries
['External relations', 'Agreements with non-member countries', 'External relations', 'Bilateral agreements with non-member countries', 'African, Caribbean and Pacific (ACP) Group of States']
['Fisheries', 'External relations', 'Agreements with non-member countries', 'External relations', 'Bilateral agreements with non-member countries', 'African, Caribbean and Pacific (ACP) Group of States']
removing Fisheries
['External relations', 'Agreements with non-member countries', 'External relations', 'Bilateral agreements with non-member countries', 'African, Caribbean and Pacific (ACP) Group of States']
['Agriculture', 'Approximation of laws and health measures', 'Animal health and zootechnics']
removing Animal health and zootechnics
['Agriculture', 'Approximation of laws 

In [0]:
Y.head()

,Eurovoc_Label
0,"[animal plague, swine, region, Austria, wild m..."
1,"[barley, intervention agency, Hungary, award o..."
2,"[common agricultural policy, common organisati..."
3,"[EAGGF Guarantee Section, closing of accounts,..."
4,"[concessionnaire, fine, infringement of Commun..."


In [0]:
import gc
gc.collect()

233

In [0]:
### comment this if previous cell is not commented

from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(Y.Eurovoc_Label)
Y = multilabel_binarizer.transform(Y.Eurovoc_Label)

In [0]:
#confirming the shape of the array

Y.shape

(19399, 3926)

In [0]:
from sklearn.model_selection import train_test_split
import gc
gc.collect()

0

**Your actual work starts from here**
Confirm Y.shape and shape of X_tfidf once


Comment the next 2 cells if the*** session keeps crashing***

In [0]:
Text = pd.merge(_stem_text_df, _label_df, on='Filename')
Text = Text.drop(columns = "Filename")
Text = Text.drop(columns = "Eurovoc_Label")


In [0]:
gc.collect()

28

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=50000, lower=True)
tokenizer.fit_on_texts(Text["Text"])
sequences = tokenizer.texts_to_sequences(Text.Text)
x = pad_sequences(sequences, maxlen=400)

Using TensorFlow backend.


In [0]:
gc.collect()

0

In [0]:
# # !pip install Wand
# import 
# from Wandb.keras import WandCallback

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
import numpy as np
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import hamming_loss
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras import optimizers
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x, Y, test_size=0.2, random_state=71)


import keras.backend as K

def mean_pred(y_true, y_pred):
  hamming_loss(y_true,y_pred)
  return K.hamming_loss(y_true,y_pred)
model = Sequential()
model.add(Embedding(50000,64 ,input_length=400))

model.add(Conv1D(64, 5, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(128, 5, padding = 'valid',activation='relu'))
model.add(Dropout(.5))
model.add(Conv1D(256, 5, padding= 'valid', activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(.5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(3926, activation='sigmoid'))



adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, 
                           epsilon=None, decay=0.0,
                           amsgrad=False)
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()
history = model.fit(x_train_tfidf, y_train_tfidf,
                    class_weight='balanced',
                    epochs=20,
                    batch_size=32)

W0619 17:17:18.316476 140210240423808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0619 17:17:18.348209 140210240423808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0619 17:17:18.355653 140210240423808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0619 17:17:18.396313 140210240423808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0619 17:17:18.416685 140210240423808 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 64)           3200000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 396, 64)           20544     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 198, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 194, 128)          41088     
_________________________________________________________________
dropout_1 (Dropout)          (None, 194, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 190, 256)          164096    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 95, 256)           0         
__________

In [0]:
#using softmax function

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
import numpy as np
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import hamming_loss
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras import optimizers
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x, Y, test_size=0.2, random_state=71)


import keras.backend as K

def mean_pred(y_true, y_pred):
  hamming_loss(y_true,y_pred)
  return K.hamming_loss(y_true,y_pred)
model = Sequential()
model.add(Embedding(50000,64 ,input_length=400))

model.add(Conv1D(64, 5, padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(128, 5, padding = 'valid',activation='relu'))
model.add(Dropout(.5))
model.add(Conv1D(256, 5, padding= 'valid', activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(.5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(3926, activation='softmax'))



adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, 
                           epsilon=None, decay=0.0,
                           amsgrad=False)
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()
history = model.fit(x_train_tfidf, y_train_tfidf,
                    class_weight='balanced',
                    epochs=20,
                    batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 400, 64)           3200000   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 396, 64)           20544     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 198, 64)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 194, 128)          41088     
_________________________________________________________________
dropout_5 (Dropout)          (None, 194, 128)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 190, 256)          164096    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 95, 256)           0         
__________

In [0]:

model.evaluate(x_test_tfidf,y_test_tfidf)
# #predicting
y_pred = model.predict(x_test_tfidf)




3880/3880 [==============================] - 1s 198us/step


Setting the top K values to 1 and others to 0

In [0]:
data_itr = 0
temp = 0
for prob_of_element in y_pred:
  k = 0
  counter = 0
  max_elements = prob_of_element.argsort()[::-1][:6]
#   print(max_elements)
  for elements in prob_of_element:
      
      if(k == max_elements[0]):
        y_pred[data_itr][k] = 1
        
      elif(k == max_elements[1]):
      
        y_pred[data_itr][k] = 1
      elif(k == max_elements[2]):
        
        y_pred[data_itr][k] = 1
      elif(k == max_elements[3]):
    

        y_pred[data_itr][k] = 1
      elif(k == max_elements[4]):
        y_pred[data_itr][k] = 1
        
      elif(k == max_elements[5]):
        y_pred[data_itr][k] = 1
     

        
#       elif(k == max_elements[6]):
        

#         y_pred[data_itr][k] = 1
      else:
        y_pred[data_itr][k] = 0
      k =k+1
  
  data_itr = data_itr+1
  if(data_itr%1000==1):
    print(data_itr)
  
  


1
1001
2001
3001


In [0]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test_tfidf,y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


0.18005954701167712

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
print("Hamming Loss Hammingloss",hamming_loss(y_test_tfidf,y_pred))
print("f1_score Micro Balanced ",f1_score(y_test_tfidf, y_pred,average='micro'))

print("f1_score Macro Balanced",f1_score(y_test_tfidf,y_pred,average='macro'))


Hamming Loss Hammingloss 0.0021121416304730294
f1_score Micro Balanced  0.2684069307380964
f1_score Macro Balanced 0.06157800492181325


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
